Benchmarking RMSF analysis speed using cloud & disk reading

Setup: 

1. Download the 90ns YiiP trajectory to the local filesystem
2. Write it into the zarrtraj format on disk
3. Upload the zarrtraj-formatted trajectory to an accessible AWS S3 bucket

In [1]:
import MDAnalysis as mda
import MDAnalysisData
import zarr

# 1
yiip = MDAnalysisData.yiip_equilibrium.fetch_yiip_equilibrium_long()
# 2
u = mda.Universe(yiip.topology, yiip.trajectory)
z = zarr.open_group("yiip.zarrtraj", mode = 'w')
with mda.writer(z, u.atoms.n_atoms,
                    n_frames=u.trajectory.n_frames,
                    # Select _ frames 
                    chunks=(10, u.trajectory.n_atoms, 3),
                    format='ZARRTRAJ') as W:

/home/law/anaconda3/envs/zarrtraj/lib/python3.10/site-packages/MDAnalysisData/base.py:34: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_string
YiiP_system_90ns_center.xtc:  62%|██████▏   | 2.31G/3.72G [15:49<06:10, 3.81MB/s]    

In [ ]:
import os 

# 3
# Must set the environmental variable using Jupyter's
# "%env KEY=VALUE"
# before running

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

source = zarr.open_group(fname, mode='r')

s3_fs = s3fs.S3FileSystem(
    # anon must be false to allow authentication
    anon=False,
    client_kwargs=dict(
        region_name='us-east-1',
    )
)

cloud_store = s3fs.S3Map(
    root=f'{BUCKET_NAME}/yiip.zarrtraj',
    s3=s3_fs,
    check=False
)

zarr.convenience.copy_store(source.store, cloud_store,
                            if_exists='replace')

cloud_dest = zarr.open_group(store=cloud_store, mode='a')
return cloud_dest